<a href="https://colab.research.google.com/github/pachterlab/BI-1C-2024/blob/main/DNA_Computing_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# This code is from the DNA Computing Simulator at https://github.com/zobront/dna-computing-simulator/tree/master?tab=readme-ov-file by Zach Obront

import random

# STEP 0: PREP
def generate_vertex_strands(V, K, pairings):
    random.seed(0)  # For reproducibility
    return [''.join([random.choice(list(pairings.keys())) for _ in range(K)]) for _ in range(V)]

def find_edge_strands(vertices, edges, K):
    edge_strands = [vertices[v1][(K//2):] + vertices[v2][:(K//2)] for (v1, v2) in edges]
    return [strand.replace(vertices[0][-(K//2):], vertices[0]).replace(vertices[-1][:(K//2)], vertices[-1]) for strand in edge_strands]

def find_complement(strand, pairings):
    return "".join([pairings[nuc] for nuc in strand])

# Initial Setup
vertex_names = ['I', 'L', 'O', 'V', 'E', 'D', 'N', 'A']
V = len(vertex_names)
edges = [(0, 1), (0, 2), (0, 5), (1, 2), (1, 7), (2, 3), (3, 2), (3, 5), (3, 4), (4, 5), (5, 1), (5, 6), (6, 7)]
pairings = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
K = 10

vertex_strands = generate_vertex_strands(V, K, pairings)
print(f"Vertex Strands Generated: {vertex_strands}")

edge_strands = find_edge_strands(vertex_strands, edges, K)
print(f"\nEdge Strands Created: {edge_strands}")

complements = [find_complement(v, pairings) for v in vertex_strands]
print(f"\nComplements Created: {complements}")

# STEP 1: CREATE RANDOM PATHS
def create_random_paths(edge_strands, K, complements, vertex_strands):
    all_edges = edge_strands * 10000
    random.shuffle(all_edges)

    path_strands = []
    growing_strand = ""

    for s in all_edges:
        if len(growing_strand) == 0:
            growing_strand += s
        else:
            target_comp = find_complement(growing_strand[-(K//2):], pairings) + find_complement(s[:(K//2)], pairings)
            if target_comp in complements:
                growing_strand += s

        if growing_strand[-K:] == vertex_strands[-1]:
            path_strands.append(growing_strand)
            growing_strand = ""

    return path_strands

path_strands = create_random_paths(edge_strands, K, complements, vertex_strands)
print(f"\nStrands Created: {len(path_strands)} ")

# STEP 2: REMOVE PATHS WITHOUT CORRECT START & END
in_and_out_strands = [path for path in path_strands if path[:K] == vertex_strands[0] and path[-K:] == vertex_strands[-1]]
print(f"\nStrands Starting at {vertex_names[0]} and Ending at {vertex_names[V-1]}: {len(in_and_out_strands)}")

# STEP 3: KEEP ONLY PATHS WITH N VERTICES
n_step_paths = [path for path in in_and_out_strands if len(path) == V * K]
print(f"\nStrands with {V} Steps: {len(n_step_paths)}")

# STEP 4: KEEP ONLY PATHS THAT TOUCH EACH VERTEX AT LEAST ONCE
def filter_paths(paths, vertex_strands, vertex_names):
    included = paths
    for i in range(len(vertex_strands)):
        included = [path for path in included if vertex_strands[i] in path]
        if i == 0 or i == len(vertex_strands) - 1:
            print(f"- Already Checked for {vertex_names[i]}")
        else:
            print(f"- Eliminating Paths Not Including {vertex_names[i]}: {len(included)} Remaining")
    return included

included = filter_paths(n_step_paths, vertex_strands, vertex_names)
print(f"Strands Including All Vertices >= Once: {len(included)}")

# STEP 5: EXTRACT ANY PATHS THAT REMAIN
def decode_solution(solution, vertex_strands, vertex_names):
    path = []
    for i in range(0, len(solution), K):
        vertex_strand = solution[i:i+K]
        vertex_num = vertex


Vertex Strands Generated: ['GGACGGCGCT', 'TCTACTCAAC', 'GACGCTGGCA', 'AAGAGCTCAT', 'TTTGAACGAC', 'CACTCGAGCT', 'CTTTACGAAT', 'TAAGCTTGCG']

Edge Strands Created: ['GGACGGCGCTTCTAC', 'GGACGGCGCTGACGC', 'GGACGGCGCTCACTC', 'TCAACGACGC', 'TCAACTAAGCTTGCG', 'TGGCAAAGAG', 'CTCATGACGC', 'CTCATCACTC', 'CTCATTTTGA', 'ACGACCACTC', 'GAGCTTCTAC', 'GAGCTCTTTA', 'CGAATTAAGCTTGCG']

Complements Created: ['CCTGCCGCGA', 'AGATGAGTTG', 'CTGCGACCGT', 'TTCTCGAGTA', 'AAACTTGCTG', 'GTGAGCTCGA', 'GAAATGCTTA', 'ATTCGAACGC']

Strands Created: 3497 

Strands Starting at I and Ending at A: 811

Strands with 8 Steps: 87
- Already Checked for I
- Eliminating Paths Not Including L: 72 Remaining
- Eliminating Paths Not Including O: 72 Remaining
- Eliminating Paths Not Including V: 72 Remaining
- Eliminating Paths Not Including E: 41 Remaining
- Eliminating Paths Not Including D: 41 Remaining
- Eliminating Paths Not Including N: 33 Remaining
- Already Checked for A
Strands Including All Vertices >= Once: 33
